In [1]:
print('hello')

hello


In [ ]:
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the model
model_name = "blaze999/Medical-NER"
print(f"📥 Loading {model_name}...\n")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

print("="*80)
print("MODEL CONFIGURATION DETAILS")
print("="*80)

# =====================================================
# 1️⃣ BASIC MODEL INFO
# =====================================================
print("\n📋 BASIC MODEL INFORMATION:")
print(f"   Model Name: {model_name}")
print(f"   Architecture: {model.config.architectures}")
print(f"   Num Labels: {model.config.num_labels}")
print(f"   Hidden Size: {model.config.hidden_size}")
print(f"   Num Attention Heads: {model.config.num_attention_heads}")
print(f"   Num Hidden Layers: {model.config.num_hidden_layers}")
print(f"   Vocab Size: {model.config.vocab_size}")

# =====================================================
# 2️⃣ LABEL MAPPINGS (ID TO LABEL & LABEL TO ID)
# =====================================================
print("\n" + "="*80)
print("LABEL MAPPINGS")
print("="*80)

id2label = model.config.id2label
label2id = model.config.label2id

print(f"\n📊 Total Labels: {len(id2label)}")
print("\n🔢 ID → LABEL Mapping:")
for label_id, label_name in sorted(id2label.items()):
    print(f"   {label_id:3d} → {label_name}")

print("\n🏷️  LABEL → ID Mapping:")
for label_name, label_id in sorted(label2id.items()):
    print(f"   {label_name:30s} → {label_id}")

# =====================================================
# 3️⃣ ENTITY TYPES (EXTRACTED FROM LABELS)
# =====================================================
print("\n" + "="*80)
print("ENTITY TYPES (BIO TAGGING SCHEME)")
print("="*80)

entity_types = {}
for label in id2label.values():
    if label.startswith('B-'):
        entity_type = label[2:]
        if entity_type not in entity_types:
            entity_types[entity_type] = {'B': [], 'I': []}
        entity_types[entity_type]['B'].append(label)
    elif label.startswith('I-'):
        entity_type = label[2:]
        if entity_type not in entity_types:
            entity_types[entity_type] = {'B': [], 'I': []}
        entity_types[entity_type]['I'].append(label)
    elif label != 'O':
        if label not in entity_types:
            entity_types[label] = {'B': [], 'I': []}

print(f"\n📌 Total Entity Types: {len(entity_types)}")
print("\n🏥 Entity Types with BIO Tags:")
for i, (entity_type, tags) in enumerate(sorted(entity_types.items()), 1):
    b_tags = tags.get('B', [])
    i_tags = tags.get('I', [])
    print(f"\n   {i:2d}. {entity_type}")
    if b_tags:
        print(f"       B-tags: {', '.join(b_tags)}")
    if i_tags:
        print(f"       I-tags: {', '.join(i_tags)}")

# =====================================================
# 4️⃣ JSON FORMAT OUTPUT
# =====================================================
print("\n" + "="*80)
print("JSON FORMAT")
print("="*80)

json_output = {
    "model_info": {
        "name": model_name,
        "architecture": model.config.architectures,
        "num_labels": model.config.num_labels,
        "hidden_size": model.config.hidden_size,
        "num_attention_heads": model.config.num_attention_heads,
        "num_hidden_layers": model.config.num_hidden_layers,
        "vocab_size": model.config.vocab_size
    },
    "label_mappings": {
        "id2label": id2label,
        "label2id": label2id
    },
    "entity_types": {
        entity_type: {
            "b_tags": tags.get('B', []),
            "i_tags": tags.get('I', [])
        }
        for entity_type, tags in sorted(entity_types.items())
    },
    "bio_scheme": {
        "description": "BIO (Begin-Inside-Outside) tagging scheme",
        "tags": {
            "B-": "Beginning of entity",
            "I-": "Inside entity (continuation)",
            "O": "Outside any entity"
        }
    }
}

print("\n📄 JSON Output:")
print(json.dumps(json_output, indent=2, ensure_ascii=False))

# Save JSON to file
with open("model_details.json", "w", encoding="utf-8") as f:
    json.dump(json_output, f, indent=2, ensure_ascii=False)
print("\n✅ JSON saved to: model_details.json")

# =====================================================
# 5️⃣ XML FORMAT OUTPUT
# =====================================================
print("\n" + "="*80)
print("XML FORMAT")
print("="*80)

# Create XML structure
root = ET.Element("medical_ner_model")

# Model Info
model_info_elem = ET.SubElement(root, "model_info")
ET.SubElement(model_info_elem, "name").text = model_name
ET.SubElement(model_info_elem, "architecture").text = str(model.config.architectures)
ET.SubElement(model_info_elem, "num_labels").text = str(model.config.num_labels)
ET.SubElement(model_info_elem, "hidden_size").text = str(model.config.hidden_size)
ET.SubElement(model_info_elem, "num_attention_heads").text = str(model.config.num_attention_heads)
ET.SubElement(model_info_elem, "num_hidden_layers").text = str(model.config.num_hidden_layers)
ET.SubElement(model_info_elem, "vocab_size").text = str(model.config.vocab_size)

# Label Mappings
label_mappings_elem = ET.SubElement(root, "label_mappings")

# ID to Label
id2label_elem = ET.SubElement(label_mappings_elem, "id2label")
for label_id, label_name in sorted(id2label.items()):
    label_elem = ET.SubElement(id2label_elem, "label", id=str(label_id))
    label_elem.text = label_name

# Label to ID
label2id_elem = ET.SubElement(label_mappings_elem, "label2id")
for label_name, label_id in sorted(label2id.items()):
    label_elem = ET.SubElement(label2id_elem, "label", name=label_name)
    label_elem.text = str(label_id)

# Entity Types
entity_types_elem = ET.SubElement(root, "entity_types")
for entity_type, tags in sorted(entity_types.items()):
    entity_elem = ET.SubElement(entity_types_elem, "entity", type=entity_type)
    
    if tags.get('B'):
        b_tags_elem = ET.SubElement(entity_elem, "b_tags")
        for tag in tags['B']:
            ET.SubElement(b_tags_elem, "tag").text = tag
    
    if tags.get('I'):
        i_tags_elem = ET.SubElement(entity_elem, "i_tags")
        for tag in tags['I']:
            ET.SubElement(i_tags_elem, "tag").text = tag

# BIO Scheme Description
bio_scheme_elem = ET.SubElement(root, "bio_scheme")
ET.SubElement(bio_scheme_elem, "description").text = "BIO (Begin-Inside-Outside) tagging scheme"
tags_elem = ET.SubElement(bio_scheme_elem, "tags")
ET.SubElement(tags_elem, "B", description="Beginning of entity")
ET.SubElement(tags_elem, "I", description="Inside entity (continuation)")
ET.SubElement(tags_elem, "O", description="Outside any entity")

# Pretty print XML
xml_string = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")

print("\n📄 XML Output:")
print(xml_string)

# Save XML to file
with open("model_details.xml", "w", encoding="utf-8") as f:
    f.write(xml_string)
print("\n✅ XML saved to: model_details.xml")

# =====================================================
# 6️⃣ TOKENIZER DETAILS
# =====================================================
print("\n" + "="*80)
print("TOKENIZER DETAILS")
print("="*80)

print(f"\n🔤 Tokenizer Type: {type(tokenizer).__name__}")
print(f"   Vocab Size: {len(tokenizer)}")
print(f"   Model Max Length: {tokenizer.model_max_length}")
print(f"   Padding Side: {tokenizer.padding_side}")
print(f"   Truncation Side: {tokenizer.truncation_side}")

# Special tokens
print(f"\n   Special Tokens:")
print(f"      PAD: {tokenizer.pad_token} (id: {tokenizer.pad_token_id})")
print(f"      UNK: {tokenizer.unk_token} (id: {tokenizer.unk_token_id})")
print(f"      CLS: {tokenizer.cls_token} (id: {tokenizer.cls_token_id})")
print(f"      SEP: {tokenizer.sep_token} (id: {tokenizer.sep_token_id})")
print(f"      MASK: {tokenizer.mask_token} (id: {tokenizer.mask_token_id})")

# =====================================================
# 7️⃣ SAMPLE ENTITY GROUPINGS FOR YOUR USE
# =====================================================
print("\n" + "="*80)
print("RECOMMENDED ENTITY GROUPINGS FOR MEDICAL APPLICATIONS")
print("="*80)

entity_groupings = {
    "MEDICATIONS": ["MEDICATION"],
    "SYMPTOMS_AND_CONDITIONS": ["SIGN_SYMPTOM", "DISEASE_DISORDER", "OUTCOME"],
    "PROCEDURES": ["DIAGNOSTIC_PROCEDURE", "THERAPEUTIC_PROCEDURE"],
    "TEMPORAL_INFO": ["DURATION", "FREQUENCY", "DATE", "TIME", "AGE"],
    "DOSAGE_INFO": ["DOSAGE", "ADMINISTRATION"],
    "MEASUREMENTS": ["LAB_VALUE", "MASS", "HEIGHT", "WEIGHT", "VOLUME", "DISTANCE"],
    "ANATOMICAL": ["BIOLOGICAL_STRUCTURE", "AREA"],
    "PATIENT_INFO": ["SEX", "AGE", "FAMILY_HISTORY", "PERSONAL_BACKGROUND", "OCCUPATION"],
    "CLINICAL_EVENTS": ["CLINICAL_EVENT", "OTHER_EVENT", "ACTIVITY"],
    "DESCRIPTIVE": ["SEVERITY", "COLOR", "SHAPE", "TEXTURE", "DETAILED_DESCRIPTION", 
                    "BIOLOGICAL_ATTRIBUTE", "QUALITATIVE_CONCEPT", "QUANTITATIVE_CONCEPT"]
}

print("\n📊 Suggested entity groupings for your application:")
for group_name, entities in entity_groupings.items():
    print(f"\n   {group_name}:")
    for entity in entities:
        if entity in entity_types:
            print(f"      ✓ {entity}")
        else:
            print(f"      ✗ {entity} (not in model)")

# Save groupings as JSON
with open("entity_groupings.json", "w", encoding="utf-8") as f:
    json.dump(entity_groupings, f, indent=2)
print("\n✅ Entity groupings saved to: entity_groupings.json")

print("\n" + "="*80)
print("✅ COMPLETE! All details exported to:")
print("   - model_details.json")
print("   - model_details.xml")
print("   - entity_groupings.json")
print("="*80)

📥 Loading blaze999/Medical-NER...



ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Check out the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [8]:
pip install torch 


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
    --------------------------------------- 2.4/109.3 MB 17.3 MB/s eta 0:00:07
   - -------------------------------------- 4.7/109.3 MB 13.2 MB/s eta 0:00:08
   -- ------------------------------------- 5.8/109.3 MB 10.3 MB/s eta 0:00:11
   -- ------------------------------------- 7.1/109.3 MB 9.4 MB/s eta 0:00:11
   --- ------------------------------------ 8.4/109.3 MB 9.0 MB/s eta 0:00:12
   --- ------------------------------------ 8.9/109.3 MB 8.2 MB/s eta 0:00:13
   --- ------------------------------------ 9.4/109.3 MB 7.1 MB/s eta 0:00:15
   --- ------------------------------------ 9.7/


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
